In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import os

In [40]:
def orb_bf_match_and_find(template_img, test_img, template_img_filename, test_img_filename, min_matches): 
    # Scale train_img to be the same size as query_img
    test_img = cv.resize(test_img, (template_img.shape[1], template_img.shape[0]))
    # Create an ORB object
    orb = cv.ORB_create()
    
    features1, des1 = orb.detectAndCompute(template_img, None)
    features2, des2 = orb.detectAndCompute(test_img, None)

    # Create Brute-Force matcher object
    bf = cv.BFMatcher(cv.NORM_HAMMING)
    matches = bf.knnMatch(des1, des2, k = 2)
    
    # Nearest neighbour ratio test to find good matches
    good = []    
    good_without_lists = []    
    matches = [match for match in matches if len(match) == 2] 
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
            good_without_lists.append(m)
         
    if len(good) >= min_matches:
        # Draw a polygon around the recognized object
        src_pts = np.float32([features1[m.queryIdx].pt for m in good_without_lists])
        dst_pts = np.float32([features2[m.trainIdx].pt for m in good_without_lists])
        
        # Get the transformation matrix
        M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC)
        pts1_transformed = cv.perspectiveTransform(np.array([src_pts]), M)
        pts1_transformed = pts1_transformed.reshape(-1, 2)

        # Compute the bounding box size
        width = 1500
        height = 1500

        # Compute the bounding box coordinates
        x_min = np.min(pts1_transformed[:, 0])
        y_min = np.min(pts1_transformed[:, 1])
        x_max = np.max(pts1_transformed[:, 0])
        y_max = np.max(pts1_transformed[:, 1])

        # Compute the center of the bounding box
        center_x = (x_min + x_max) / 2
        center_y = (y_min + y_max) / 2

        # Compute the upper left corner of the bounding box
        x = center_x - width / 2
        y = center_y - height / 2

        # Draw the bounding box
        cv.rectangle(test_img, (int(x), int(y)), (int(x + width), int(y + height)), (0, 255, 0), 2)
    print('Total matches found: {}'.format(len(good)))  
    result_img = cv.drawMatchesKnn(template_img, features1, test_img, features2, good, None, flags = 4)
    cv.putText(result_img, 'Total matches found: {}'.format(len(good)), (200, 200), cv.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 2, cv.LINE_AA)
    cv.imwrite('output/orb/bf/{}_and_{}.jpg'.format(template_img_filename, test_img_filename), result_img)

def orb_bf_match_with_images_folder(path, min_matches):
    template = cv.imread(path)
    template_filename = os.path.splitext(os.path.basename(path))[0]
    for filename in os.listdir('output/orb/bf'):
        if filename.startswith(template_filename):
            os.remove('output/orb/bf/' + filename)
    for filename in os.listdir('images'):
        test_img = cv.imread('images/' + filename)
        test_img_filename = os.path.splitext(filename)[0]
        orb_bf_match_and_find(template, test_img, template_filename, test_img_filename, min_matches)

In [41]:
# Running matching with remote control template
orb_bf_match_with_images_folder('templates/remote-1.jpg', 10)
orb_bf_match_with_images_folder('templates/remote-2.jpg', 10)

Total matches found: 6
Total matches found: 8
Total matches found: 9
Total matches found: 17
Total matches found: 6
Total matches found: 8
Total matches found: 12
Total matches found: 21
Total matches found: 8
Total matches found: 6
Total matches found: 8
Total matches found: 22
Total matches found: 3
Total matches found: 49
Total matches found: 16
Total matches found: 14
Total matches found: 29
Total matches found: 13
Total matches found: 9
Total matches found: 20
Total matches found: 24
Total matches found: 6
Total matches found: 5
Total matches found: 15
Total matches found: 14
Total matches found: 2
Total matches found: 3
Total matches found: 5
Total matches found: 9
Total matches found: 3
Total matches found: 33
Total matches found: 12
Total matches found: 6
Total matches found: 25


In [43]:
# Running matching with earphone template
orb_bf_match_with_images_folder('templates/earphone-1.jpg', 25)
orb_bf_match_with_images_folder('templates/earphone-2.jpg', 25)

Total matches found: 15
Total matches found: 23
Total matches found: 35
Total matches found: 27
Total matches found: 14
Total matches found: 17
Total matches found: 20
Total matches found: 11
Total matches found: 13
Total matches found: 20
Total matches found: 18
Total matches found: 15
Total matches found: 31
Total matches found: 12
Total matches found: 40
Total matches found: 17
Total matches found: 11
Total matches found: 17
Total matches found: 20
Total matches found: 26
Total matches found: 31
Total matches found: 7
Total matches found: 14
Total matches found: 26
Total matches found: 16
Total matches found: 13
Total matches found: 12
Total matches found: 23
Total matches found: 17
Total matches found: 18
Total matches found: 11
Total matches found: 25
Total matches found: 8
Total matches found: 12


In [44]:
# Running matching with elephant template
orb_bf_match_with_images_folder('templates/elephant-1.jpg', 8)
orb_bf_match_with_images_folder('templates/elephant-2.jpg', 8)

Total matches found: 14
Total matches found: 11
Total matches found: 15
Total matches found: 15
Total matches found: 11
Total matches found: 4
Total matches found: 7
Total matches found: 8
Total matches found: 8
Total matches found: 10
Total matches found: 7
Total matches found: 10
Total matches found: 9
Total matches found: 8
Total matches found: 7
Total matches found: 4
Total matches found: 6
Total matches found: 10
Total matches found: 6
Total matches found: 8
Total matches found: 6
Total matches found: 6
Total matches found: 4
Total matches found: 4
Total matches found: 6
Total matches found: 2
Total matches found: 6
Total matches found: 7
Total matches found: 7
Total matches found: 5
Total matches found: 12
Total matches found: 4
Total matches found: 12
Total matches found: 9


In [42]:
orb_bf_match_with_images_folder('templates/pokemon-1.jpg', 20)

Total matches found: 72
Total matches found: 11
Total matches found: 12
Total matches found: 8
Total matches found: 30
Total matches found: 13
Total matches found: 13
Total matches found: 12
Total matches found: 4
Total matches found: 115
Total matches found: 7
Total matches found: 21
Total matches found: 10
Total matches found: 6
Total matches found: 9
Total matches found: 12
Total matches found: 7


In [2]:
def sift_bf_match_and_find(template_img, test_img, template_img_filename, test_img_filename, min_matches): 
    # Scale train_img to be the same size as query_img
    test_img = cv.resize(test_img, (template_img.shape[1], template_img.shape[0]))
    # Create an SIFT object
    sift = cv.SIFT_create()
    
    features1, des1 = sift.detectAndCompute(template_img, None)
    features2, des2 = sift.detectAndCompute(test_img, None)

    # Create Brute-Force matcher object
    bf = cv.BFMatcher(cv.NORM_L2)
    matches = bf.knnMatch(des1, des2, k = 2)
    
    # Nearest neighbour ratio test to find good matches
    good = []    
    good_without_lists = []    
    matches = [match for match in matches if len(match) == 2] 
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
            good_without_lists.append(m)
         
    if len(good) >= min_matches:
        # Draw a polygon around the recognized object
        src_pts = np.float32([features1[m.queryIdx].pt for m in good_without_lists])
        dst_pts = np.float32([features2[m.trainIdx].pt for m in good_without_lists])
        
        # Get the transformation matrix
        M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC)
        pts1_transformed = cv.perspectiveTransform(np.array([src_pts]), M)
        pts1_transformed = pts1_transformed.reshape(-1, 2)

        # Compute the bounding box size
        width = 1500
        height = 1500

        # Compute the bounding box coordinates
        x_min = np.min(pts1_transformed[:, 0])
        y_min = np.min(pts1_transformed[:, 1])
        x_max = np.max(pts1_transformed[:, 0])
        y_max = np.max(pts1_transformed[:, 1])

        # Compute the center of the bounding box
        center_x = (x_min + x_max) / 2
        center_y = (y_min + y_max) / 2

        # Compute the upper left corner of the bounding box
        x = center_x - width / 2
        y = center_y - height / 2

        # Draw the bounding box
        cv.rectangle(test_img, (int(x), int(y)), (int(x + width), int(y + height)), (0, 255, 0), 2)
    print('Total matches found: {}'.format(len(good)))
    result_img = cv.drawMatchesKnn(template_img, features1, test_img, features2, good, None, flags = 4)
    cv.putText(result_img, 'Total matches found: {}'.format(len(good)), (200, 200), cv.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 2, cv.LINE_AA)
    cv.imwrite('output/sift/bf/{}_and_{}.jpg'.format(template_img_filename, test_img_filename), result_img)

def sift_bf_match_with_images_folder(path, min_matches):
    template = cv.imread(path)
    template_filename = os.path.splitext(os.path.basename(path))[0]
    for filename in os.listdir('output/sift/bf'):
        if filename.startswith(template_filename):
            os.remove('output/sift/bf/' + filename)
    for filename in os.listdir('images'):
        test_img = cv.imread('images/' + filename)
        test_img_filename = os.path.splitext(filename)[0]
        sift_bf_match_and_find(template, test_img, template_filename, test_img_filename, min_matches)

In [3]:
# Running matching with remote control template
sift_bf_match_with_images_folder('templates/remote-1.jpg', 350)
sift_bf_match_with_images_folder('templates/remote-2.jpg', 350)

Total matches found: 298
Total matches found: 262
Total matches found: 202
Total matches found: 459
Total matches found: 377
Total matches found: 275
Total matches found: 301
Total matches found: 429
Total matches found: 328
Total matches found: 371
Total matches found: 426
Total matches found: 364
Total matches found: 336
Total matches found: 264
Total matches found: 308
Total matches found: 219
Total matches found: 253
Total matches found: 1422
